In [39]:
import sympy as sp

In [55]:
def getMetric(lineElement, coordSystem="Cartesian", subs=None, overrideConst = False): #the override lets the code run faster if you know for sure your line element will work out
    if coordSystem not in ["Cartesian", "PlanePolar", "SphericalPolar", "CylindricalPolar"]:
        raise ValueError("Unknown coordinate system")

    lineElement=sp.expand(lineElement)
    coords = (t, x, y, z)

    dim = len(coords)
    g = sp.zeros(dim)

    for mu in range(dim):
        for nu in range(dim):
            coeff = lineElement.coeff(sp.diff(coords[mu]) * sp.diff(coords[nu]))
            if mu != nu and coeff != 0:
                g[mu, nu] = coeff.subs(subs) / 2
            else:
                g[mu, nu] = coeff.subs(subs)
                
    # Check for unexpected terms in the line element
    if not overrideConst:
        reconstructed_line_element = sum(g[i, j] * sp.diff(coords[i]) * sp.diff(coords[j]) for i in range(dim) for j in range(dim))
        if sp.simplify(lineElement.subs(subs) - reconstructed_line_element) != 0:
            raise ValueError("Line element contains terms that are not pure differentials of the coordinates used")
    return g

In [53]:

vs, sigma, R, lam = sp.symbols('v_s sigma R lambda')
t = sp.Function('t')(lam)
x = sp.Function('x')(lam)
y = sp.Function('y')(lam)
z = sp.Function('z')(lam)

dt=sp.diff(t)
dx=sp.diff(x)
dy=sp.diff(y)
dz=sp.diff(z)

# for now, define constants as symbols
xs, r, f_r = sp.symbols("x_s r f_r")

lineElement = -dt**2 + (dx - vs*f_r*dt)**2 + dy**2 + dz**2

# order of substitutions matter!!!
subs= [
    (xs, vs*t), # since its steady state
    (f_r, (sp.tanh(sigma * (r + R)) - sp.tanh(sigma * (r - R))) / (2 * sp.tanh(sigma * R))),
    (r, sp.sqrt((x - xs)**2 + y**2 + z**2)),
]

metric=getMetric(lineElement, "Cartesian", subs)
metric

Matrix([
[v_s**2*(-tanh(sigma*(-R + sqrt((-x_s + x(lambda))**2 + y(lambda)**2 + z(lambda)**2))) + tanh(sigma*(R + sqrt((-x_s + x(lambda))**2 + y(lambda)**2 + z(lambda)**2))))**2/(4*tanh(R*sigma)**2) - 1, -v_s*(-tanh(sigma*(-R + sqrt((-x_s + x(lambda))**2 + y(lambda)**2 + z(lambda)**2))) + tanh(sigma*(R + sqrt((-x_s + x(lambda))**2 + y(lambda)**2 + z(lambda)**2))))/(2*tanh(R*sigma)), 0, 0],
[            -v_s*(-tanh(sigma*(-R + sqrt((-x_s + x(lambda))**2 + y(lambda)**2 + z(lambda)**2))) + tanh(sigma*(R + sqrt((-x_s + x(lambda))**2 + y(lambda)**2 + z(lambda)**2))))/(2*tanh(R*sigma)),                                                                                                                                                                                    1, 0, 0],
[                                                                                                                                                                                               0,                             

## subbing into previously used christoffel symbol calculator

In [54]:
metric_inv = metric.inv()

# defining a partial
def partial_derivative(matrix, var):
    """ This function returns the matrix of partial derivatives """
    return sp.Matrix(matrix.shape[0], matrix.shape[1], lambda i,j: sp.diff(matrix[i, j], var))

# derivatives of metric tensor
partial_t = partial_derivative(metric, t)
partial_x = partial_derivative(metric, x)
partial_y = partial_derivative(metric, y)
partial_z = partial_derivative(metric, z)

print("start computation")
# computing the symbols using the metric equation
christoffel_symbols = [[[0 for i in range(4)] for j in range(4)] for k in range(4)]
for lambda_ in range(4):
    for mu in range(4):
        for nu in range(4):
            christoffel_symbols[lambda_][mu][nu] = 1/2 * (
                metric_inv[lambda_, 0] * (partial_x[mu, nu] + partial_x[nu, mu] - partial_t[mu, nu]) +
                metric_inv[lambda_, 1] * (partial_t[mu, nu] + partial_t[nu, mu] - partial_x[mu, nu]) +
                metric_inv[lambda_, 2] * (partial_y[mu, nu] + partial_y[nu, mu] - partial_y[mu, nu]) +
                metric_inv[lambda_, 3] * (partial_z[mu, nu] + partial_z[nu, mu] - partial_z[mu, nu])
            ).simplify()
            print(f"l: {lambda_} mu: {mu} nu{nu}")
            print(christoffel_symbols[lambda_][mu][nu])

christoffel_symbols[0][1][0]  # Displaying Γ^t_{tx} as an example

start computation
l: 0 mu: 0 nu0
-0.125*sigma*v_s**2*(x_s - x(lambda))*(v_s*(tanh(sigma*(R - sqrt(x_s**2 - 2*x_s*x(lambda) + x(lambda)**2 + y(lambda)**2 + z(lambda)**2))) + tanh(sigma*(R + sqrt(x_s**2 - 2*x_s*x(lambda) + x(lambda)**2 + y(lambda)**2 + z(lambda)**2)))) - 4*tanh(R*sigma))*(tanh(sigma*(R - sqrt((x_s - x(lambda))**2 + y(lambda)**2 + z(lambda)**2))) + tanh(sigma*(R + sqrt((x_s - x(lambda))**2 + y(lambda)**2 + z(lambda)**2))))*(tanh(sigma*(R - sqrt((x_s - x(lambda))**2 + y(lambda)**2 + z(lambda)**2)))**2 - tanh(sigma*(R + sqrt((x_s - x(lambda))**2 + y(lambda)**2 + z(lambda)**2)))**2)/(sqrt((x_s - x(lambda))**2 + y(lambda)**2 + z(lambda)**2)*tanh(R*sigma)**3)
l: 0 mu: 0 nu1
0.125*sigma*v_s*(x_s - x(lambda))*(v_s*(tanh(sigma*(R - sqrt(x_s**2 - 2*x_s*x(lambda) + x(lambda)**2 + y(lambda)**2 + z(lambda)**2))) + tanh(sigma*(R + sqrt(x_s**2 - 2*x_s*x(lambda) + x(lambda)**2 + y(lambda)**2 + z(lambda)**2)))) - 4*tanh(R*sigma))*(tanh(sigma*(R - sqrt((x_s - x(lambda))**2 + y(lambda)**2 

0.125*sigma*v_s*(x_s - x(lambda))*(v_s*(tanh(sigma*(R - sqrt(x_s**2 - 2*x_s*x(lambda) + x(lambda)**2 + y(lambda)**2 + z(lambda)**2))) + tanh(sigma*(R + sqrt(x_s**2 - 2*x_s*x(lambda) + x(lambda)**2 + y(lambda)**2 + z(lambda)**2)))) - 4*tanh(R*sigma))*(tanh(sigma*(R - sqrt((x_s - x(lambda))**2 + y(lambda)**2 + z(lambda)**2)))**2 - tanh(sigma*(R + sqrt((x_s - x(lambda))**2 + y(lambda)**2 + z(lambda)**2)))**2)/(sqrt((x_s - x(lambda))**2 + y(lambda)**2 + z(lambda)**2)*tanh(R*sigma)**2)